# EDA and Feature Engineering - Noise Events Prediction Model

This notebook will be used to explore the available data, where possible features for the predictive model will be evaluated.

In [3]:
import polars as pl
import numpy as np

## Meteo Data Exploration

In [23]:
df_meteo = pl.read_parquet('meteo_data.parquet')
df_meteo.head()

DATEUTC,ID,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,Date,Year,Month,Day,Hour,Minute,LC_RAD60,LC_TEMP_QCL0,LC_TEMP_QCL1,LC_TEMP_QCL2,LC_TEMP_QCL3
str,str,i64,f64,i64,i64,f64,f64,i64,f64,str,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64
"""2022-01-01 00:…","""LC-002""",92,11.78,38,0,0.0,0.0,-169,0.43,"""2022-01-01""",2022,1,1,0,10,0,13.11,13.11,13.0515,13.048027
"""2022-01-01 00:…","""LC-002""",92,11.73,37,0,0.0,0.0,-170,0.33,"""2022-01-01""",2022,1,1,0,20,0,13.01,13.01,12.9515,12.985849
"""2022-01-01 00:…","""LC-002""",92,11.73,38,0,0.0,0.0,-167,0.46,"""2022-01-01""",2022,1,1,0,30,0,13.0,13.0,12.9415,12.950322
"""2022-01-01 00:…","""LC-002""",92,11.72,37,0,0.0,0.0,-160,0.52,"""2022-01-01""",2022,1,1,0,40,0,13.0,13.0,12.9415,12.94955
"""2022-01-01 00:…","""LC-002""",92,11.72,38,0,0.0,0.0,-166,0.51,"""2022-01-01""",2022,1,1,0,50,0,13.0,13.0,12.9415,12.952268


In [26]:
df_meteo['LC_DAILYRAIN'].value_counts()

LC_DAILYRAIN,counts
f64,u32
0.118,3
0.137,33
0.062,11
0.123,1
0.139,29
0.119,121
0.135,21
0.131,1
0.06,19


## Noise Events Exploration

In [3]:
df_events = pl.read_parquet('noise_events.parquet')
df_events.head()

#object_id,description,result_timestamp,noise_event_laeq_model_id,noise_event_laeq_model_id_unit,noise_event_laeq_primary_detected_certainty,noise_event_laeq_primary_detected_certainty_unit,noise_event_laeq_primary_detected_class,noise_event_laeq_primary_detected_class_unit
i64,str,str,i64,str,i64,str,str,str
255439,"""MP 01: Naamses…","""28/02/2022 08:…",null,null,null,"""%""",null,null
255439,"""MP 01: Naamses…","""28/02/2022 13:…",null,null,null,"""%""",null,null
255439,"""MP 01: Naamses…","""28/02/2022 16:…",null,null,null,"""%""",null,null
255439,"""MP 01: Naamses…","""28/02/2022 19:…",null,null,null,"""%""",null,null
255439,"""MP 01: Naamses…","""28/02/2022 20:…",null,null,null,"""%""",null,null


In [9]:
df_events = pl.read_parquet('noise_events.parquet')

In [15]:
df_events['noise_event_laeq_primary_detected_class'].is_null().sum()

3822

In [19]:
df_events['result_timestamp'][0]

'28/02/2022 08:27:21.737'

In [20]:
df_levels = pl.read_parquet('noise_levels_parquets/April.parquet', n_rows=50)

In [21]:
df_levels.head()

#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit
i64,str,str,f64,str,f64,str,f64,str,f64,str
255439,"""MP 01: Naamses…","""01/04/2022 00:…",62.5,"""dB(A)""",59.3,"""dB(A)""",64.56,"""dB(C)""",78.13,"""dB(C)"""
255439,"""MP 01: Naamses…","""01/04/2022 00:…",63.3,"""dB(A)""",61.3,"""dB(A)""",65.35,"""dB(C)""",78.09,"""dB(C)"""
255439,"""MP 01: Naamses…","""01/04/2022 00:…",61.4,"""dB(A)""",59.1,"""dB(A)""",64.18,"""dB(C)""",76.63,"""dB(C)"""
255439,"""MP 01: Naamses…","""01/04/2022 00:…",58.9,"""dB(A)""",56.6,"""dB(A)""",63.58,"""dB(C)""",75.74,"""dB(C)"""
255439,"""MP 01: Naamses…","""01/04/2022 00:…",59.6,"""dB(A)""",57.2,"""dB(A)""",63.32,"""dB(C)""",77.86,"""dB(C)"""
